In [ ]:
import dlt
from pyspark.sql.functions import col, max as spark_max, count, current_timestamp

@dlt.table(
    name="flood_dev.gold.latest_alerts",
    comment="Latest flood severity per Environment Agency region.",
    table_properties={"quality": "gold"}
)
def latest_alerts():
    df = dlt.read("flood_dev.silver.floods_clean")

    latest = (
        df.groupBy("region_name")
          .agg(
              spark_max("event_time").alias("latest_event_time"),
              spark_max("severity").alias("current_severity"),
              spark_max("alert_description").alias("current_alert"),
              count("*").alias("record_count")
          )
          .withColumn("last_refreshed", current_timestamp())
    )

    return latest
